In [14]:
!wget http://spades.bioinf.spbau.ru/students/4/ecoli_400K_err_1.fastq.gz -O ~/ngs_institute/data/ecoli_400K_err_1.fastq.gz
!wget http://spades.bioinf.spbau.ru/students/4/ecoli_400K_err_2.fastq.gz -O ~/0_ngs_institute/data/ecoli_400K_err_2.fastq.gz
!wget http://spades.bioinf.spbau.ru/students/4/MG1655-K12.first400K.fasta -O ~/ngs_institute/data/MG1655-K12.first400K.fasta
!gunzip ~/ngs_institute/data/ecoli_400K_err_1.fastq.gz
!gunzip ~/0_ngs_institute/data/ecoli_400K_err_2.fastq.gz    
    

--2019-04-01 19:57:21--  http://spades.bioinf.spbau.ru/students/4/ecoli_400K_err_2.fastq.gz
Resolving spades.bioinf.spbau.ru (spades.bioinf.spbau.ru)... 194.85.238.21
Connecting to spades.bioinf.spbau.ru (spades.bioinf.spbau.ru)|194.85.238.21|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 122167351 (117M) [application/x-gzip]
Saving to: <</Users/dashabalashova/0_ngs_institute/data/ecoli_400K_err_2.fastq.gz>>

/Users/dashabalasho 100%[===================>] 116.51M  1.18MB/s    in 95s     

2019-04-01 19:55:19 (1.23 MB/s) - <</Users/dashabalashova/0_ngs_institute/data/ecoli_400K_err_2.fastq.gz>> saved [122167351/122167351]



In [138]:
!spades.py --only-error-correction -1 ~/ngs_institute/data/ecoli_400K_err_1.fastq -2 ~/ngs_institute/data/ecoli_400K_err_2.fastq -o ~/ngs_institute/HW4/output

!bowtie2-build ~/ngs_institute/data/MG1655-K12.first400K.fasta MG1655-K12_index
!bowtie2 -x MG1655-K12_index -1 ~/0_ngs_institute/data/ecoli_400K_err_1.fastq -2 ~/0_ngs_institute/data/ecoli_400K_err_2.fastq -S ecoli.sam
!bowtie2 -x MG1655-K12_index -1 ~/ngs_institute/HW4/output/corrected/ecoli_400K_err_1.00.0_0.cor.fastq -2 ~/ngs_institute/HW4/output/corrected/ecoli_400K_err_2.00.0_0.cor.fastq -S ecoli_cor.sam



In [10]:
!samtools view -S -b ecoli.sam | samtools sort > ecoli.sorted.bam
!samtools index ecoli.sorted.bam

!samtools view -S -b ecoli_cor.sam > ecoli_cor.bam
!samtools sort ecoli_cor.bam > ecoli_cor.sorted.bam
!samtools index ecoli_cor.sorted.bam


In [32]:
!bowtie2 -x MG1655-K12_index -1 ecoli_1_1000sample.fastq -2 ecoli_2_1000sample.fastq -S ecoli_1000sample.sam
!bowtie2 -x MG1655-K12_index -1 ecoli_1_cor_1000sample.fastq -2 ecoli_2_cor_1000sample.fastq -S ecoli_cor_1000sample.sam


250 reads; of these:
  250 (100.00%) were paired; of these:
    46 (18.40%) aligned concordantly 0 times
    199 (79.60%) aligned concordantly exactly 1 time
    5 (2.00%) aligned concordantly >1 times
    ----
    46 pairs aligned concordantly 0 times; of these:
      0 (0.00%) aligned discordantly 1 time
    ----
    46 pairs aligned 0 times concordantly or discordantly; of these:
      92 mates make up the pairs; of these:
        55 (59.78%) aligned 0 times
        37 (40.22%) aligned exactly 1 time
        0 (0.00%) aligned >1 times
89.00% overall alignment rate


In [31]:
!samtools flagstat ecoli_1000sample.sam
!samtools view -S -b ecoli_1000sample.sam > ecoli.bam
!samtools sort ecoli.bam > ecoli.sorted.bam
!samtools index ecoli.sorted.bam

500 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
499 + 0 mapped (99.80% : N/A)
500 + 0 paired in sequencing
250 + 0 read1
250 + 0 read2
498 + 0 properly paired (99.60% : N/A)
498 + 0 with itself and mate mapped
1 + 0 singletons (0.20% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


In [33]:
!samtools flagstat ecoli_cor_1000sample.sam
!samtools view -S -b ecoli_cor_1000sample.sam > ecoli_cor.bam
!samtools sort ecoli_cor.bam > ecoli_cor.sorted.bam
!samtools index ecoli_cor.sorted.bam

500 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
445 + 0 mapped (89.00% : N/A)
500 + 0 paired in sequencing
250 + 0 read1
250 + 0 read2
408 + 0 properly paired (81.60% : N/A)
408 + 0 with itself and mate mapped
37 + 0 singletons (7.40% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


In [34]:
!samtools view ecoli_cor.sorted.bam | fgrep -w -f ecoli_cor_names.txt
!samtools view ecoli.sorted.bam | fgrep -w -f ecoli_names.txt

fgrep: ecoli_cor_names.txt: No such file or directory
samtools view: writing to standard output failed: Broken pipe
samtools view: error closing standard output: -1
fgrep: ecoli_names.txt: No such file or directory
samtools view: writing to standard output failed: Broken pipe
samtools view: error closing standard output: -1


In [52]:
!samtools view -F 4 ecoli_cor.sorted.bam | cut -f1 | sort -u > ecoli_cor_names.txt
!samtools view -F 4 ecoli.sorted.bam | cut -f1 | sort -u > ecoli_names.txt


In [62]:
import pysam

set1 = set(line.strip()[:-2] for line in open('ecoli_cor_names.txt'))

set2 = set(line.strip() for line in open('ecoli_names.txt'))

intersection = set1.intersection(set2)



  # read names go here
bam = pysam.AlignmentFile('ecoli.sorted.bam')
obam = pysam.AlignmentFile('ecoli_intersection.bam', "w", template=bam)

for b in bam.fetch(until_eof=True):
    if b.query_name in intersection:
        obam.write(b)
bam.close()
obam.close()

bam = pysam.AlignmentFile('ecoli_cor.sorted.bam')
obam = pysam.AlignmentFile('ecoli_cor_intersection.bam', "w", template=bam)

for b in bam.fetch(until_eof=True):
    if b.query_name[:-2] in intersection:
        obam.write(b)
bam.close()
obam.close()

!samtools sort -n ecoli_intersection.bam -o ecoli_intersection_sorted.bam
!samtools sort -n ecoli_cor_intersection.bam -o ecoli_cor_intersection_sorted.bam


In [131]:
bamfile1 = pysam.AlignmentFile('ecoli_intersection_sorted.bam', 'rb')
bamfile2 = pysam.AlignmentFile('ecoli_cor_intersection_sorted.bam', 'rb')
NN=0
true_rem = 0
false_rem = 0
true_positive = 0
true_negative = 0
false_positive = 0
false_negative = 0
for read in zip(bamfile1,bamfile2):
    NN+=1
    if read[0].cigarstring!=None and read[1].cigarstring!=None:
        N = min(len(read[0].get_reference_sequence()), len(read[1].get_reference_sequence()), len(read[0].query_sequence), len(read[1].query_sequence))
        for n in range(0, N):
            
            if read[0].get_reference_sequence()[n].islower()==1: #error
                if read[1].query_sequence[n]=='N':
                    true_rem+=1
                    #print('true_rem =', true_rem)
                elif read[1].get_reference_sequence()[n].islower()==0:
                    true_positive+=1
                    #print('true positive =', true_positive)
                else:
                    false_negative+=1
                    #print('false negative =', false_negative)
            
            else:
                if read[1].get_reference_sequence()[n].islower()==0:
                    true_negative+=1
                elif read[1].query_sequence[n]=='N':
                    false_rem+=1
                    #print('false_rem =', false_rem)
                else:
                    false_positive+=1
                    #print('false positive =', false_positive)
        #print(true_rem+true_positive+false_negative+true_negative+false_rem+false_positive)

                    
#import time
#t=time.time()
print('true remove =', true_rem)
print('false remove =', false_rem)
print('true positive =', true_positive)
print('true negative =', true_negative)
print('false positive =', false_positive)
print('false negative =', false_negative)
#print((time.time()-t)/60)



true remove = 463866
false remove = 3647270
true positive = 685132
true negative = 215197058
false positive = 2021
false negative = 220097


In [137]:
bamfile1 = pysam.AlignmentFile('ecoli_intersection_sorted.bam', 'rb')
bamfile2 = pysam.AlignmentFile('ecoli_cor_intersection_sorted.bam', 'rb')
NN=0
true_rem = 0
false_rem = 0
true_positive = 0
true_negative = 0
false_positive = 0
false_negative = 0
for read in zip(bamfile1,bamfile2):
    NN+=1
    if NN<10 and read[0].cigarstring!=None and read[1].cigarstring!=None:
    #if N<10 and read[0].query_sequence!=read[1].query_sequence and read[0].cigarstring!=None and read[1].cigarstring!=None:
        #print(len(read[0].get_reference_sequence()))
        #print(len(read[0].get_reference_sequence()))
        N = min(len(read[0].get_reference_sequence()), len(read[1].get_reference_sequence()), len(read[0].query_sequence), len(read[1].query_sequence))
        #print(N)
        K = 0
        for n in range(0, N):
            
            if read[0].get_reference_sequence()[n].islower()==1: #error
                if read[1].query_sequence[n]=='N':
                    true_rem+=1
                    
                    #print('true_rem =', true_rem)
                elif read[1].get_reference_sequence()[n].islower()==0:
                    true_positive+=1
                    
                    #print('true positive =', true_positive)
                else:
                    false_negative+=1
                    #print('false negative =', false_negative)
            
            else:
                if read[1].get_reference_sequence()[n].islower()==0:
                    true_negative+=1
                elif read[1].query_sequence[n]=='N':
                    false_rem+=1
                    #print('false_rem =', false_rem)
                else:
                    false_positive+=1
                    K=1
        if K==1:
            print(read[0].query_name)
            print(read[0].query_sequence)
            print(read[0].get_reference_sequence())
            print(read[1].query_name)
            print(read[1].query_sequence)
            print(read[1].get_reference_sequence())